In [ ]:
import treelite
import tl2cgen
import sklearn.datasets as datasets
import xgboost as xgb
import numpy as np
import subprocess
import random
import subprocess
import matplotlib.pyplot as plt
import platform
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from ucimlrepo import fetch_ucirepo 

In [ ]:
datasets = [fetch_ucirepo(id=17), fetch_ucirepo(id=31), fetch_ucirepo(id=545), fetch_ucirepo(id=109), fetch_ucirepo(id=159), fetch_ucirepo(id=94), fetch_ucirepo(id=42), fetch_ucirepo(id=572)]

In [ ]:
datasets_names = ['Breast Cancer', 'Covertype', 'Rice', 'Wine', 'Magic','Spam', 'Glass', 'Bankruptcy']
tree_depths = [3, 5, 10, 15, 20]
nr_root_nodes = 20
nr_validation_reuses = 1000
nr_instances_validation = 500

In [ ]:
def FreeTayK(path: str):
    with open(path, 'r+') as f:
        data = f.readlines()
        f.seek(0, 0)
        f.write('#include "quantize.c"\n' + ''.join(data[2:]))
    
def create_c_array_string(array):
    array_string = "{\n"
    array_string += ",\n".join("{" + ", ".join(str(x) for x in row) + "}" for row in array)
    array_string += "\n}"
    return array_string

def count_decimals(number):
    str_num = str(number)
    if '.' not in str_num:
        return 0
    return len(str_num) - str_num.index('.') - 1

def beg_c_file_str(nr_datasets, nr_treedepths) :
    a = """
#include <time.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <windows.h>
#include "dll_paths.h"


typedef void (*PredictFunction)(union Entry*, int, float*);
int main() {
char times["""+str(nr_datasets*nr_treedepths*30)+"""] = "";
int k=0;
int nr_treedepths = """+str(nr_treedepths)+""";
    """
    return a

def mid_c_file_str(i, nr_results, reuses):
    a = """
//////////////////////////////////////////////

    for (int i = (0+(k*3))*nr_treedepths; i < (3 +(k*3))*nr_treedepths; i++) {
        HINSTANCE hDLL = LoadLibraryA(dll_paths[i]);
        if (hDLL == NULL) {
            return 1;
        }
        PredictFunction predict = (PredictFunction) GetProcAddress(hDLL, "predict");
        clock_t start = clock();
        for (int z = 0; z < """+str(reuses)+"""; z++) {
            for (int j = 0; j < sizeof(validation_instances"""+str(i)+""")/sizeof(validation_instances"""+str(i)+"""[0]); j++) {
                float result["""+str(nr_results)+"""]={0.0f};
                predict(validation_instances"""+str(i)+"""[j], 0, &result);
            }
        }
        FreeLibrary(hDLL);
        clock_t end = clock();
        double elapsed_time = ((double) (end - start)) / CLOCKS_PER_SEC;
        sprintf(times + strlen(times), "%6.3f\\n", elapsed_time);
    }
k++;

//////////////////////////////////////////////
"""
    return a

def end_c_file_str() :
    a = """
FILE *file = fopen("times.txt", "w");
if (file == NULL) {
    printf("Could not open file for writing.\\n");
    return 1;
}
fprintf(file, times);
fclose(file);
return 0;
}
    """
    return a

def compile_c_file(c_file_path):
    # gcc is the compiler, -o is used to specify the output file
    # 'output' is the name of the output file
    command = ['gcc', c_file_path, '-o', 'output']

    try:
        # Run the command
        subprocess.check_call(command)
        print("Compilation successful.")
    except subprocess.CalledProcessError:
        print("Compilation failed.")

def run_exe_file(exe_file_path):
    try:
        # Run the executable file
        subprocess.check_call(exe_file_path)
        print("Execution successful.")
    except subprocess.CalledProcessError:
        print("Execution failed.")





datasets = [fetch_ucirepo(id=17), fetch_ucirepo(id=31), fetch_ucirepo(id=545), fetch_ucirepo(id=109), fetch_ucirepo(id=159), fetch_ucirepo(id=94), fetch_ucirepo(id=42), fetch_ucirepo(id=572)]
datasets_names = ['Breast Cancer', 'Covertype', 'Rice', 'Wine', 'Magic','Spam', 'Glass', 'Bankruptcy']
tree_depths = [3, 5, 10, 15, 20]
nr_root_nodes = 20

header_file_str = r'const char *dll_paths[] = {'
c_file_str = beg_c_file_str(len(datasets), len(tree_depths))
dll_file_paths = []
nr_results = []
nr_features = []
nr_instances =[]
validation_instances = []
validation_instances_str = r''
nr_validation_reuses = 1000

for i, dataset in enumerate(datasets):
    for depth in tree_depths:

        # Train models 
        X = dataset.data.features 
        y = dataset.data.targets 
        le = LabelEncoder()
        y = le.fit_transform(y)

        X.columns = X.columns.str.replace('[', '')
        X.columns = X.columns.str.replace(']', '')
        X.columns = X.columns.str.replace('<', '')

        dtrain = xgb.DMatrix(X, label=y)
        params = {"max_depth": depth, "eta": 0.1, "objective": "multi:softprob", "eval_metric": "mlogloss", "num_class": len(np.unique(y))}
        bst = xgb.train(params, dtrain, num_boost_round=nr_root_nodes, evals=[(dtrain, 'train')])
        model = treelite.Model.from_xgboost(bst)

        path_no_param = "./Trees/model"+str(i)+"N"+str(depth)
        path_flint = "./Trees/model"+str(i)+"F"+str(depth)
        path_quantize = "./Trees/model"+str(i)+"Q"+str(depth)


        tl2cgen.generate_c_code(model, dirpath=path_no_param, params={})
        tl2cgen.generate_c_code(model, dirpath=path_flint, params={"flint" : 1})
        tl2cgen.generate_c_code(model, dirpath=path_quantize, params={"quantize": 1})

        # Generate DLLs
        path_no_param_dll = path_no_param + '/predict.dll'
        path_flint_dll = path_flint + '/predict.dll'
        path_quantize_dll = path_quantize + '/predict.dll'

        dll_file_paths.append(path_no_param_dll)
        dll_file_paths.append(path_flint_dll)
        dll_file_paths.append(path_quantize_dll)

        FreeTayK(path_quantize + '/main.c')
        
        # Compile the generated C code into DLLs
        subprocess.run(['gcc', '-shared', '-o', path_no_param_dll, path_no_param + '/main.c']),
        subprocess.run(['gcc', '-shared', '-o', path_flint_dll, path_flint + '/main.c']),
        subprocess.run(['gcc', '-shared', '-o', path_quantize_dll, path_quantize + '/main.c'])

    # Generate random validation instances with decimal precision equal to the maximum decimal precision of the training data
    min_values_feature = []
    max_values_feature = []
    nr_decimals = np.zeros((X.shape[0], X.shape[1]))
    max_decimals = []

    nr_results = len(np.unique(y))
    nr_features.append(X.shape[1])
    nr_instances.append(X.shape[0])
    for k in range(min(X.shape[0],1000)):
        for j in range(X.shape[1]):
            nr_decimals[i][j] = count_decimals(X.iloc[k, j])

    for k in range(X.shape[1]):
        max_decimals.append(max(nr_decimals[:, k]))


    for k in range(X.shape[1]):
        min_values_feature.append(X.iloc[:, k].min())
        max_values_feature.append(X.iloc[:, k].max())

    validation_instances = np.zeros((nr_instances_validation, X.shape[1]))
    for j in range(nr_instances_validation):
        for k in range(X.shape[1]):
            validation_instances[j][k] = round(random.uniform(min_values_feature[k], max_values_feature[k]), int(max_decimals[k]))
    


    c_file_str += mid_c_file_str(i, nr_results, nr_validation_reuses)
    validation_instances_str += 'float validation_instances'+str(i)+'[][' + str(X.shape[1]) + '] = ' + create_c_array_string(validation_instances) + ';\n'

header_file_str += ', '.join([f'"{path}"' for path in dll_file_paths]) + '};\n'

print (header_file_str)
header_file_str += validation_instances_str



with open('dll_paths.h', 'w') as f:
    f.write(header_file_str)


c_file_str += end_c_file_str()
with open('generated_speedTest.c', 'w') as f:
    f.write(c_file_str)



In [ ]:
compile_c_file('generated_speedTest.c')
run_exe_file('output')
with open('times.txt', 'r') as f:
    times_content = f.read()
times = np.fromstring(times_content, sep='\n')
times


In [ ]:
# Read out times.txt without compiling and running the C code again
with open('times.txt', 'r') as f:
    times_content = f.read()
times = np.fromstring(times_content, sep='\n')
times

In [ ]:
# Figure scaling so that 6 bar groups are 10 wide
plt.figure(figsize=((len(times)/3)*(10/6), 5))

x = np.arange(0, len(times), 3)

width = 2

# Create the bar chart for each group with different colors
colors = ['red', 'green', 'blue']
labels = ['No parameters', 'Flint', 'Quantize']
for i in range(3):
    plt.bar(x + i*(width/3), times[i::3], color=colors[i], width=width/3, label=labels[i])

names = [f"{name}{depth}" for name in datasets_names for depth in tree_depths]

architecture = platform.architecture()

# Set the x-tick labels to the group names and remove the x-axis
plt.xticks(x-0.5 + width/2, names)
plt.tick_params(axis='x', length=0)

plt.ylabel('Seconds')
plt.title('Inference times')
plt.text(0, -0.1, 'Validation instances per set: '+str(nr_instances_validation)+'\nValidation set reuses: '+str(nr_validation_reuses)+'\nNumber of root nodes: '+str(nr_root_nodes)+'\nSystem info: '+str(architecture), ha='left', va='top', transform=plt.gca().transAxes)

# Display the legend
plt.legend()

plt.show()

In [ ]:
# Figure scaling so that 6 bar groups are 10 wide
plt.figure(figsize=((len(times)/(3*len(tree_depths)))*(10/6), 5))

speedups_flint = times[0::3]/times[1::3]
speedups_quantize = times[0::3]/times[2::3]

width = 2
speedups = np.ones(len(datasets_names)*3)
for i in range(len(datasets_names)):
    speedups[i*3+1] = np.mean(speedups_flint[i*len(tree_depths):i*len(tree_depths)+len(tree_depths)])
    speedups[i*3+2] = np.mean(speedups_quantize[i*len(tree_depths):i*len(tree_depths)+len(tree_depths)])


x = np.arange(0, len(speedups), 3)

# Create the bar chart for each group with different colors
colors = ['red', 'green', 'blue']
labels = ['No parameters', 'Flint', 'Quantize']
for i in range(3):
    plt.bar(x + i*(width/3), speedups[i::3], color=colors[i], width=width/3, label=labels[i])


architecture = platform.architecture()

# Set the x-tick labels to the group names and remove the x-axis
plt.xticks(x-0.5 + width/2, datasets_names)
plt.tick_params(axis='x', length=0)

plt.ylabel('Speedup factor')
plt.title('Average speedup factors per dataset')
plt.text(0, -0.1, 'Validation instances per set: '+str(nr_instances_validation)+'\nValidation set reuses: '+str(nr_validation_reuses)+'\nNumber of root nodes: '+str(nr_root_nodes)+'\nSystem info: '+str(architecture), ha='left', va='top', transform=plt.gca().transAxes)

# Display the legend
plt.legend()

plt.show()

In [ ]:
times_no_parameters = [np.mean(times[i*3::len(tree_depths)*3]) for i in range(len(tree_depths))]
times_flint = [np.mean(times[i*3+1::len(tree_depths)*3]) for i in range(len(tree_depths))]
times_quantize = [np.mean(times[i*3+2::len(tree_depths)*3]) for i in range(len(tree_depths))]

architecture = platform.architecture()
# Create the line plot for each group with different colors
plt.plot(range(len(tree_depths)), times_no_parameters, color='red', marker='*', label='No parameters')
plt.plot(range(len(tree_depths)), times_flint, color='green', marker='^', label='Flint')
plt.plot(range(len(tree_depths)), times_quantize, color='blue', marker='s', label='Quantize')

# Set the x-tick labels to the values in tree_depths
plt.xticks(range(len(tree_depths)), tree_depths)

plt.xlabel('Maximum tree depth')
plt.ylabel('Time [s]')
plt.title('Average inference time per tree depth')

plt.text(-0.1, -0.15, 'Datasets used:\n'+str(datasets_names)+'.\nValidation instances per set: '+str(nr_instances_validation)+'\nValidation set reuses: '+str(nr_validation_reuses)+'\nSystem info: '+str(architecture), ha='left', va='top', transform=plt.gca().transAxes)

# Display the legend
plt.legend()

plt.show()

In [ ]:
speedups_flint, speedups_quantize = [], []  
factors_flint = [times[i*3]/times[i*3+1] for i in range(len(datasets_names)*len(tree_depths))]
factors_quantize = [times[i*3]/times[i*3+2] for i in range(len(datasets_names)*len(tree_depths))]

for i in range(len(tree_depths)):
    speedups_flint.append(np.mean(factors_flint[i::len(tree_depths)]))
    speedups_quantize.append(np.mean(factors_quantize[i::len(tree_depths)]))


architecture = platform.architecture()
# Create the line plot for each group with different colors
plt.plot(range(len(tree_depths)), np.ones(len(tree_depths)), color='red', marker='*', label='No parameters')
plt.plot(range(len(tree_depths)), speedups_flint, color='green', marker='^', label='Flint')
plt.plot(range(len(tree_depths)), speedups_quantize, color='blue', marker='s', label='Quantize')

# Set the x-tick labels to the values in tree_depths
plt.xticks(range(len(tree_depths)), tree_depths)

plt.xlabel('Maximum tree depth')
plt.ylabel('Speedup factor')
plt.title('Average speedup factor per tree depth')

plt.text(-0.1, -0.15, 'Datasets used:\n'+str(datasets_names)+'.\nValidation instances per set: '+str(nr_instances_validation)+'\nValidation set reuses: '+str(nr_validation_reuses)+'\nSystem info: '+str(architecture), ha='left', va='top', transform=plt.gca().transAxes)

# Display the legend
plt.legend()

plt.show()

In [ ]:
# WIP
times_per_nr_features= []

indices = sorted(range(len(nr_features)), key=nr_features.__getitem__)
sorted_nr_features = sorted(nr_features)


times_no_parameters = [[np.mean(times[i*len(tree_depths)*3:i*len(tree_depths)*3+len(tree_depths)*3:3]) for i in range(len(datasets))][j] for j in indices]
times_flint = [[np.mean(times[i*len(tree_depths)*3+1:i*len(tree_depths)*3+len(tree_depths)*3+1:3]) for i in range(len(datasets))][j] for j in indices]
times_quantize = [[np.mean(times[i*len(tree_depths)*3+2:i*len(tree_depths)*3+len(tree_depths)*3+2:3]) for i in range(len(datasets))][j] for j in indices]



# Create the line plot for each group with different colors
plt.plot(range(len(sorted_nr_features)), times_no_parameters, color='red', marker='*', label='No parameters')
plt.plot(range(len(sorted_nr_features)), times_flint, color='green', marker='^', label='Flint')
plt.plot(range(len(sorted_nr_features)), times_quantize, color='blue', marker='s', label='Quantize')

# Set the x-tick labels to the values in tree_depths
plt.xticks(range(len(sorted_nr_features)), sorted_nr_features)

plt.xlabel('Number of features')
plt.ylabel('Time [s]')
plt.title('Average inference time per number of features')

plt.text(-0.1, -0.15, 'Datasets used: \n'+str(datasets_names)+'.\nValidation instances per set: '+str(nr_instances_validation)+'\nValidation set reuses: '+str(nr_validation_reuses)+'\nSystem info: '+str(architecture), ha='left', va='top', transform=plt.gca().transAxes)

# Display the legend
plt.legend()

plt.show()